# 한총 계절 장바구니 분석

In [1]:
#데이터셋 가져오기
#order 컬럼명 바꾸기 - 숫자로 되어 있는 부분에 Order (purchase event) 입력
import pandas as pd
#카테고리 전처리
import re
# 장바구니 분석
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import os

### 0. 경로 설정

In [2]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총/계절' #input("작업 파일 경로를 입력하세요")
os.chdir(input_path)
path=os.getcwd()
print(path)

#폴더 생성
#x = dt.datetime.now()
year='2020'
file_name=f'계절분석결과_{year}'

if not os.path.exists(file_name):
    os.mkdir(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총\계절


In [40]:
season_name={0: '봄', 1: '여름', 2:'가을',3:'겨울'}

def frozen_convert(sets) : 
        return [list(sets)]

def season_filter(data, month1, month2, month3):
    return data[data['Month'].str.contains(month1) |data['Month'].str.contains(month2)|data['Month'].str.contains(month3)]

### 1. 한총 데이터 로드

In [41]:
df= pd.read_clipboard(sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321492 entries, 0 to 321491
Data columns (total 3 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Month                      321492 non-null  object
 1   Model Code (v41) (evar41)  321437 non-null  object
 2   Order (purchase event)     321492 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 7.4+ MB


In [42]:
for i in df.columns:
    if i=='Model Code (v41) (evar41)': 
        df=df.rename(columns={'Model Code (v41) (evar41)':'Model Code (v41)'})
print(df.columns)
# 날짜 보기
print('날짜 보기: ',df['Month'].unique())

print('원하는 날짜보기: ',df[df['Month'].str.endswith('20')]['Month'].unique())

Index(['Month', 'Model Code (v41)', 'Order (purchase event)'], dtype='object')
날짜 보기:  ['Jan-20' 'Feb-20' 'Mar-20' 'Apr-20' 'May-20' 'Jun-20' 'Jul-20' 'Aug-20'
 'Sep-20' 'Oct-20' 'Nov-20' 'Dec-20' 'Jan-21' 'Feb-21' 'Mar-21' 'Apr-21'
 'May-21' 'Jun-21' 'Jul-21' 'Aug-21' 'Sep-21' 'Oct-21' 'Nov-21' 'Dec-21'
 'Jan-22' 'Feb-22' 'Mar-22' 'Apr-22' 'May-22' 'Jun-22' 'Jul-22' 'Aug-22']
원하는 날짜보기:  ['Jan-20' 'Feb-20' 'Mar-20' 'Apr-20' 'May-20' 'Jun-20' 'Jul-20' 'Aug-20'
 'Sep-20' 'Oct-20' 'Nov-20' 'Dec-20']


### 해당 날짜 선정 (na값 제외)

In [43]:
df=df[df['Month'].str.endswith('20')]
print('원하는 날짜 행: ',df.shape[0])
df=df.dropna(axis=0)
print('빈행제거후: ',df.shape[0])

원하는 날짜 행:  62621
빈행:  62597


In [45]:
# 계절별 전체 
DF_SEASON=df.groupby('Month', as_index=False).sum()
DF_spring=season_filter(DF_SEASON,'Mar','Apr','May')['Order (purchase event)'].sum()
DF_summer=season_filter(DF_SEASON,'Jun','Jul','Aug')['Order (purchase event)'].sum()
DF_autumn=season_filter(DF_SEASON,'Sep','Oct','Nov')['Order (purchase event)'].sum()
DF_winter=season_filter(DF_SEASON,'Dec','Jan','Feb')['Order (purchase event)'].sum()
print(DF_SEASON)
print('******* 계절별 전체 ********')
print('봄: ',DF_spring,'여름: ',DF_summer, '가을: ',DF_autumn,'겨울: ',DF_winter)


order_2=df[df['Model Code (v41)'].str.contains(',')]
order_spring=season_filter(order_2,'Mar','Apr','May')['Order (purchase event)'].sum()
order_summer=season_filter(order_2,'Jun','Jul','Aug')['Order (purchase event)'].sum()
order_autumn=season_filter(order_2,'Sep','Oct','Nov')['Order (purchase event)'].sum()
order_winter=season_filter(order_2,'Dec','Jan','Feb')['Order (purchase event)'].sum()

print('******* 계절별 2개이상 구매 시  ********')
print('봄: ',order_spring,'여름: ',order_summer, '가을: ',order_autumn,'겨울: ',order_winter)

print('******** 비중 ***************')
print('봄: ',round((order_spring/DF_spring)*100,2),'여름: ',round((order_summer/DF_summer)*100,2), '가을: ',round((order_autumn/DF_autumn)*100,2),'겨울: ',round((order_winter/DF_winter)*100,2))

     Month  Order (purchase event)
0   Apr-20                    6752
1   Aug-20                   23746
2   Dec-20                   50249
3   Feb-20                   13681
4   Jan-20                    3036
5   Jul-20                    6675
6   Jun-20                    8559
7   Mar-20                   11914
8   May-20                    6608
9   Nov-20                  114815
10  Oct-20                   12095
11  Sep-20                   13585
******* 계절별 전체 ********
봄:  25274 여름:  38980 가을:  140495 겨울:  66966
******* 계절별 2개이상 구매 시  ********
봄:  1579 여름:  9240 가을:  2647 겨울:  4262
******** 비중 ***************
봄:  6.25 여름:  23.7 가을:  1.88 겨울:  6.36


### 쓸모 있는 데이터만 : ,들어간거 + order 0 아닌것들(,있는데 왜 0...)

In [47]:
df=df[df['Order (purchase event)']!= 0]
print('0 제외: ',df.shape[0])
df=df[df['Model Code (v41)'].str.contains(',')]
print('콤마 들어간거 :',df.shape[0])
#df=df.reset_index(drop=True)

0 제외:  15535
콤마 들어간거 : 4850


In [48]:
spring=season_filter(df,'Mar','Apr','May')
summer=season_filter(df,'Jun','Jul','Aug')
autumn=season_filter(df,'Sep','Oct','Nov')
winter=season_filter(df,'Dec','Jan','Feb')

### 2. 카테고리 파일 로드

In [10]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Category2D        37 non-null     object
 1   Category2D_Value  37 non-null     object
dtypes: object(2)
memory usage: 720.0+ bytes


In [12]:
for season_index, season in enumerate([spring, summer, autumn, winter]):

    season=season.reset_index(drop=True)
    
    #전체 SKU 하나의 리스트로
    a = season['Model Code (v41)'].tolist()
    b = ','.join(a) #리스트 문자열로
    c = b.split(',') #문자열 개별 문자로 리스트화 
    model = list(set(c)) #중복 제거

    #전체 SKU 담을 딕셔너리
    dic_category = {}

    for m in model :
        flag = 1
        for i in range(len(rex)):
            r = re.compile(rex.Category2D_Value[i])
            if r.search(m):
                dic_category[m] = rex.Category2D[i]
                flag = 0
                continue
        if flag :
            dic_category[m] = '기타'

    #데이터셋 만들기
    dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

    for i in range(len(season)) :
        sku = season['Model Code (v41)'][i]
        revised = sku.split(",")
        code = []
        for j in range(len(revised)) :
            code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
        
        for k in range(season['Order (purchase event)'][i]) :   #order 수만큼 담기
            dataset.append(code) 
    a = pd.DataFrame(dataset)


    #장바구니 분석
    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset) #True / False
    season2 = pd.DataFrame(te_ary, columns=te.columns_)
    itemsets_data = apriori(season2, min_support=0.00001, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
    rules = association_rules(itemsets_data, metric='lift', min_threshold=1).sort_values('lift',ascending=False).reset_index(drop=True)
    rules['count']=(rules['support']*season['Order (purchase event)'].sum()).astype(int)

    x = dt.datetime.now()

    num_conq=1
    #N=4


    for num_ant in [1,2,3]:
        df = rules[rules.apply(lambda x:True if len(x.antecedents) == int(num_ant) else False, axis=1) & rules.apply(lambda x:True if len(x.consequents) == int(num_conq) else False, axis=1)]
        df = df.sort_values('confidence', ascending=False)
        #df = df.round(N)
        df[['antecedents','consequents']] =df[['antecedents','consequents']].applymap(lambda x: frozen_convert(x))
        df['antecedents']=df['antecedents'].astype(str).apply(lambda x: x.replace('[[','['))
        df['antecedents']=df['antecedents'].apply(lambda x: x.replace(']]',']'))
        df['consequents']=df['consequents'].astype(str).apply(lambda x: x.replace('[[','['))
        df['consequents']=df['consequents'].apply(lambda x: x.replace(']]',']'))
        df.to_excel(f'{path}/{file_name}/{year}_한총 {season_name.get(season_index)} 분석결과파일_{num_ant}.xlsx')


    